<a href="https://colab.research.google.com/github/fantasticfears/seq2seq-lab/blob/master/2-log-linear-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The aim of this lab is to develop a Language Model based on the log linear model.

Again, we use the data from `data/iwslt-en-de-preprocessed.tar.gz`. The dataset we are going to use comes from German-English translation task in IWSLT campaign.

I'm not in favor of this excises nor the symbols used in the tutorial.

## Prepare data

In [0]:
import tarfile
import re

from math import log
from collections import Counter, defaultdict
from itertools import chain

import numpy as np
import pandas
from scipy.special import softmax

from processdata import get_vocabs, tokenize, START_SYMBOL, STOP_SYMBOL

In [0]:
VOCABS = get_vocabs("data/ngram/train.txt", "data/ngram/valid.txt", "data/ngram/test.txt")

In [0]:
def generate_mapping_from_vocabs(vocab):
  with open('data/loglinear/vocab_mapping.txt', 'w') as f:
    for i, v in enumerate(vocab):
      f.write(f"{v}\t{i}\n")

generate_mapping_from_vocabs(VOCABS)

In [0]:
def load_mapping_from_vocabs(file, convert_to_int=True):
  mapping = {}
  with open(file, 'r') as f:
    for l in f:
      k, v = l.rstrip().split('\t')
      if convert_to_int:
        mapping[k] = int(v)
      else:
        mapping[k] = v
  return mapping

def transform_data(mapping, file):
  seqs = []
  with open(file, 'r') as f:
    for l in f:
      line_mapping = [mapping[t] for t in tokenize(l)]
      seqs.append(line_mapping)
  
  return seqs

def writing_seq_idx(filename, seqs):
  with open(filename, 'w') as f:
    for l in seqs:
      f.write(" ".join(l) + "\n")

In [0]:
mapping = load_mapping_from_vocabs('data/loglinear/vocab_mapping.txt', convert_to_int=False)

for i in ["train", "valid", "test"]:
  writing_seq_idx(f"data/loglinear/{i}.txt", transform_data(mapping, f"data/ngram/{i}.txt"))

## Design feature functions

Let's just use one-hot vector.
How many features do we need?

### Vanilla feature functions

Here we go. How large it is to represent all features?

In [0]:
def feature_fn(seq, vocab_size):
  def gen():
    for s in seq:
      feature = np.zeros(vocab_size, dtype=np.float32)
      feature[s] = 1
      yield feature
  
  return np.vstack([f for f in gen()]).reshape(-1)

In [0]:
# play with some parameters
vocab_size = 5
feature_length = 2
seq = (3,2)

In [0]:
print(feature_fn(seq, vocab_size), feature_fn(seq, vocab_size).shape)

[0. 0. 0. 1. 0. 0. 0. 1. 0. 0.] (10,)


### Score function and loss function
Now we need to set up the computation.

In [0]:
def build_model(vocab_size, feature_length):
  W = np.empty((vocab_size, feature_length*vocab_size), dtype=np.float32)
  b = np.empty((vocab_size), dtype=np.float32)
  return W, b

In [0]:
W, b = build_model(vocab_size, feature_length) 

In [0]:
def score(x, W, b):
  return np.dot(W, x) + b

In [0]:
# try it
x = feature_fn(seq, vocab_size)
s = score(x, W, b)
print(s)

[2.5249703e-29 2.0938583e-38 1.3213755e-19 1.0000000e+00 1.9836774e-38]


In [0]:
def probability(s):
  res = softmax(s)
  res[res==0.0] = 1e-45 # a quick fix on clipping
  return res

In [0]:
p = probability(s)
print(p)

[0.14884758 0.14884758 0.14884758 0.40460965 0.14884758]


In [0]:
def loss(estimated, expected):
  mask = expected>=1.0
  return sum(-np.ma.log(estimated[mask]))

In [0]:
target = 2
target_onehot = feature_fn((target,), vocab_size)
loss(p, target_onehot)

1.9048324823379517

### calculate derivatives

Firstly, the compuation is here:

$$
\begin{aligned} \boldsymbol{x} &=\phi\left(e_{t-m+1}^{t-1}\right) \\ \boldsymbol{s} &=\sum_{\left\{j : x_{j} !=0\right\}} W _{:,j} x_{j}+\boldsymbol{b} \\ \boldsymbol{p} &=\operatorname{softmax}(\boldsymbol{s}) \\ \ell &=-\log \boldsymbol{p}_{e_{t}} \end{aligned}
$$

We can get:

$$
\begin{aligned} \frac{d \ell\left(e_{t-n+1}^{t}, W, \boldsymbol{b}\right)}{d \boldsymbol{b}} &=\boldsymbol{p}-\text { onehot }\left(e_{t}\right) \\ \frac{d \ell\left(e_{t-n+1}^{t}, W, \boldsymbol{b}\right)}{d W_{\cdot, j}} &=x_{j}\left(\boldsymbol{p}-\text { onehot }\left(e_{t}\right)\right) \end{aligned}
$$

In [0]:
db = p - target_onehot
dW = ((x >= 1.0).astype(int).reshape(-1, 1) * db).T
print(db, b.shape, db.shape)
print(dW, W.shape, dW.shape)

[ 0.14884758  0.14884758 -0.8511524   0.40460965  0.14884758] (5,) (5,)
[[ 0.          0.          0.          0.14884758  0.          0.
   0.          0.14884758  0.          0.        ]
 [ 0.          0.          0.          0.14884758  0.          0.
   0.          0.14884758  0.          0.        ]
 [-0.         -0.         -0.         -0.85115242 -0.         -0.
  -0.         -0.85115242 -0.         -0.        ]
 [ 0.          0.          0.          0.40460965  0.          0.
   0.          0.40460965  0.          0.        ]
 [ 0.          0.          0.          0.14884758  0.          0.
   0.          0.14884758  0.          0.        ]] (5, 10) (5, 10)


In [0]:
def sgd_opt_step(W, b, dW, db, lr=1e-4):
  W -= lr * dW
  b -= lr * db
  return W, b

In [0]:
# See how it changes
print(b)
W, b = sgd_opt_step(W, b, dW, db, lr=1e-4)
print(b)

[0. 0. 0. 1. 0.]
[-1.4884758e-05 -1.4884758e-05  8.5115236e-05  9.9995953e-01
 -1.4884758e-05]


### Optimize our computation

In fact, we are already optimize the computation in optimization steps.
The things for our model is that we don't need to generate a one-hot vector for a feature.
We only needs to pluck a column instead of dot product when we perform transformation.

In [0]:
def col_feature_fn(seq, vocab_size):
  def gen():
    acc = 0
    for s in seq:
      yield s + acc
      acc += vocab_size
  
  return [f for f in gen()]

def score(x, W, b):
  # we don't have broadcast according to formula. It's different than engineering process
  return np.sum(W[:,x], axis=1) + b

In [0]:
x = col_feature_fn(seq, vocab_size)
score(x, W, b)

array([-4.4654273e-05, -4.4654273e-05,  2.5534572e-04,  9.9987859e-01,
       -4.4654273e-05], dtype=float32)

## Putting them together
Just show case which part might be really slow

In [0]:
%load_ext line_profiler

def step():
  feature_length = 2
  vocab_size = len(VOCABS)

  seq = (1,2)
  target = 3
  W, b = build_model(vocab_size, feature_length) 
  x = col_feature_fn(seq, vocab_size)
  s = score(x, W, b)
  p = probability(s)
  target_onehot = feature_fn((target,), vocab_size)
  l = loss(p, target_onehot)
  db = p - target_onehot
  dW = np.zeros(W.shape[1], dtype=np.float32) # we spent a lot of time here though!
  dW[np.array(x)] = 1.0
  dW = (dW.reshape(-1, 1) * db).T
  sgd_opt_step(W, b, dW, db, lr=1e-4)

%lprun -f step step()

In [0]:
def pad_sequence(
    sequence,
    n,
    mapping
):
  LEFT_PAD_SYMBOL = int(mapping['<s>'])
  RIGHT_PAD_SYMBOL = int(mapping['</s>'])
  return chain((LEFT_PAD_SYMBOL,) * (n - 1), iter(sequence), (RIGHT_PAD_SYMBOL,) * (n - 1))

def ngram(sequence, n, mapping):
  sequence = pad_sequence(sequence, n, mapping)

  history = []
  while n > 1:
    try:
      next_item = next(sequence)
    except StopIteration:
      # no more data, terminate the generator
      return
    history.append(next_item)
    n -= 1
  
  for item in sequence:
    history.append(mapping[item])
    yield tuple(history)
    del history[0]

def train(filename, mapping, epoch=10):
  vocab_size = len(VOCABS)
  feature_length = 2
  W, b = build_model(vocab_size, feature_length) 
  
  with open(filename, 'r') as f:
    for e in range(epoch):
      dW = np.zeros(W.shape, np.float32)
      db = np.zeros(b.shape, np.float32)
      loss_val = 0.0
      i = 0
      
      for l in f:
        for seq in ngram(map(int, l.rstrip().split(' ')), feature_length+1, mapping):
          x = col_feature_fn(seq[:-1], vocab_size)
          s = score(x, W, b)
          p = probability(s)

          target_onehot = feature_fn((seq[-1],), vocab_size)
          loss_val += loss(p, target_onehot)

          db_val = p - target_onehot
          db += db_val
          dW[:,x] += (dW[:,x].T + db_val.T).T

        i += 1
        if i >= 100:
          i = 0
          print(f"loss: {loss_val}")
          loss_val = 0.0
          W, b = sgd_opt_step(W, b, dW, db, lr=1e-4)
          dW = np.zeros(W.shape, np.float32)
          db = np.zeros(b.shape, np.float32)
      f.seek(0)
  return W, b

In [0]:
W, b = train("data/loglinear/train.txt", mapping)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: overflow encountered in add


loss: 23338.241806030273


/usr/local/lib/python3.7/site-packages/scipy/special/_logsumexp.py:215: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


ValueError: invalid literal for int() with base 10: ''

This is not efficient code at all. Since we take all data into generate vocabs,
so we don't need to deal with unknown words. But if we want to make it more fruitful,
we can generate vocabs based on training data (which is also correct).

And be noted, this is not common practice to write computation like this.
It's hard to vectorize. It's hard to use broadcast from numpy (which asks us to align the last dimension).
Why is that? (Thing about memory layout)